# Introduction to Plotly

Let's start by importing the necessary libraries. 

In the past Plotly had offline and online modes. When in online mode, all the graphs you created would be saved in your Plotly account online. But there was a limit of graphs you could create for free, so you either had to constantly delete your graphs from your account or pay. The alternative was to switch to offline mode, so that your graphs were only displayed in your Jupyter Notebook and are not saved online. 

Starting from Plotly version 4.0 released in July 2019, the offline mode is used by default, and the online functionality is moved to a separate package called `chart-studio` (more on Plotly version 4.0 [here](https://medium.com/plotly/plotly-py-4-0-is-here-offline-only-express-first-displayable-anywhere-fc444e5659ee)). So make sure you are using tha latest version of Plotly and in this case you don't need to specify that you want to use offline mode explicitly.

In [ ]:
# check your plotly version
import plotly 
plotly.__version__

To upgrade to the latest Plotly version, uncomment and run the following cell:

In [ ]:
#!pip install --upgrade plotly

In [ ]:
# pandas and numpy to work with datasets
import pandas as pd
import numpy as np

# plotly imports for Plotly visualizations
import plotly.graph_objs as go
import plotly.express as px

## Dataset exploration

We will use data about OECD countries meat consumption from [Data World](https://data.world/oecd/meat-consumption). 


**Summary** 

Meat consumption is related to living standards, diet, livestock production and consumer prices, as well as macroeconomic uncertainty and shocks to GDP. Compared to other commodities, meat is characterised by high production costs and high output prices. Meat demand is associated with higher incomes and a shift - due to urbanisation - to food consumption changes that favour increased proteins from animal sources in diets. While the global meat industry provides food and a livelihood for billions of people, it also has significant environmental and health consequences for the planet. This indicator is presented for beef and veal, pig, poultry, and sheep. Meat consumption is measured in thousand tonnes of carcass weight (except for poultry expressed as ready to cook weight) and in kilograms of retail weight per capita. Carcass weight to retail weight conversion factors are: 0.7 for beef and veal, 0.78 for pigmeat, and 0.88 for both sheep meat and poultry meat.



In [ ]:
# loading dataset into pandas dataframe
df = pd.read_csv('meat_consumption_clean.csv')

In [ ]:
# print out first 5 rows of the dataset
df.head()

In [ ]:
# get the number of rows and columns in the dataset
df.shape

In [ ]:
# get a sense of values in each column of the dataset
df.describe(include='all')

In [ ]:
# get the list of unique countries
df.LOCATION.unique()

In [ ]:
# get the list of unique meat types
df.SUBJECT.unique()

In [ ]:
# get the list of unique measures
df.MEASURE.unique()

Since there are 2 different measures of meat consumption in this dataset, let's separate it into 2 datasets: 

- `df_total` with total meat consumption in thousand tonnes of carcass weight
- `df_per_cap` with meat consumption measured in kilogramms per capita of retail weight

In [ ]:
# split the dataset into 2 based on the measure
df_total =  df.query('MEASURE == "THND_TONNE"')
df_per_cap = df.query('MEASURE == "KG_CAP"')

## Building graphs with Plotly

The native syntax of Plotly includes 3 main components: 

- **Data**

The Data object defines the data that will be displayed in the graph. 

- **Layout**

The Layout object defines all additional features of the graph, like title, axis titles, etc. 

- **Figure** 

The Figure object brings together the Data and the Layout and creates the graph to be plotted. 


You can find full Plotly documentation for Python [here](https://plot.ly/python/).

Let's start by building a simple line chart summarizing the evolution of total meat consumption by year.

In [ ]:
# remove OECD, BRICS
df_total_clean = df_total.query('LOCATION != "OECD" & LOCATION != "BRICS"')

# group by year
df_total_year = df_total_clean.groupby('TIME').Value.sum().round(0).reset_index()
df_total_year

In [ ]:
# save the split datasets to csv for further use
df_total_clean.to_csv('df_total.csv', index=False)
df_per_cap.to_csv('df_per_cap.csv', index=False)

This would be the full syntax for defining the figure as one object at once:

In [ ]:
fig = go.Figure(
        data=[
            go.Scatter(x=df_total_year.TIME, 
                       y=df_total_year.Value,
                       mode='lines')
        ],
        layout=go.Layout(
            title=dict(text='Total Meat Consumption Evolution'),
            xaxis=dict(title='Year'),
            yaxis=dict(title='Meat consumption, thousand tonnes')
        )
    )
fig.show()

In [ ]:
print(fig)

This is a more step by step way to define the same graph, using methods `add_trace` and `layout.update` on the figure object.

In [ ]:
# create figure object
fig = go.Figure()
# create a data trace 
fig.add_trace(go.Scatter(x=df_total_year.TIME, 
                         y=df_total_year.Value,
                         mode='lines'))
# edit the layout
fig.update_layout(title='Total Meat Consumption Evolution',
                   xaxis_title='Year',
                   yaxis_title='Meat consumption, thousand tonnes')
fig.show()

Now let's build a multiple lines chart, adding a split by type of meat to the previous graph.

In [ ]:
# create a dataframe with meat consumption by type by year
df_year_type = df_total_clean.groupby(['TIME', 'SUBJECT']).Value.sum().round(0).reset_index()
df_year_type.head()

To plot the results easily, let's pivot this dataframe, so that each column represents a type of meat and each row represents a year and on the intersection we have the actual value of thousands of tonnes of meat consumed.

In [ ]:
# pivot the dataframe using pivot_table function
df_year_type_pivot = df_year_type.pivot_table(index='TIME', columns='SUBJECT', values='Value').reset_index()
df_year_type_pivot.head()

In [ ]:
fig = go.Figure()
# create a data trace for each type of meat and add them to figure one by one
fig.add_trace(go.Scatter(x=df_year_type_pivot.TIME, 
                         y=df_year_type_pivot.BEEF,
                         mode='lines+markers',
                         name='beef'))
fig.add_trace(go.Scatter(x=df_year_type_pivot.TIME, 
                         y=df_year_type_pivot.PIG,
                         mode='lines+markers',
                         name='pig'))
fig.add_trace(go.Scatter(x=df_year_type_pivot.TIME, 
                         y=df_year_type_pivot.POULTRY,
                         mode='lines+markers',
                         name='poultry'))
fig.add_trace(go.Scatter(x=df_year_type_pivot.TIME, 
                         y=df_year_type_pivot.SHEEP,
                         mode='lines+markers',
                         name='sheep'))
# edit the layout
fig.update_layout(title='Meat Consumption Evolution by Meat Type',
                   xaxis_title='Year',
                   yaxis_title='Meat consumption, thousand tonnes')
fig.show()

## Building Plotly graphs with Plotly Express

The main advantage of using Plotly Express over native Plotly syntax is that it is much less verbose. You can find full Plotly Express documentation [here](https://plot.ly/python/plotly-express/).

Let's recreate the same line chart using Plotly Express. We will use `line` function for this.

In [ ]:
px.line(df_total_year, x="TIME", y="Value", title='Total Meat Consumption Evolution')

To add axes titles, we need to use the `update_layout` method.

In [ ]:
px.line(df_total_year, x="TIME", y="Value", title='Total Meat Consumption Evolution').\
update_layout(xaxis_title='Year', yaxis_title='Meat consumption, thousand tonnes')

In case of multiple lines graph, Plotly Express syntax will also be much less verbose than native Plotly syntax. We just need to pass the name of the column by which we want to split the lines to the `color` attribute.

In [ ]:
px.line(df_year_type, x='TIME', y='Value', color='SUBJECT', title='Meat Consumption Evolution by Meat Type').\
update_layout(xaxis_title='Year', yaxis_title='Meat consumption, thousand tonnes').\
update_traces(mode='lines+markers')

### PRACTICE TIME

Create a bar chart showing total meat consumption in kg per capita per country (where each country is a bar) in year 2018 using either Plotly native syntax or Plotly Express. 

**BONUS POINTS!** Turn the bar chart into stacked bar chart by adding a split by type of meat.

[Here](https://plot.ly/python/bar-charts/) is Plotly documentation for creating bar charts.
[Here](https://plot.ly/python/plotly-express/#visualize-distributions) is Plotly Express documentation for creating bar charts.

Remember to filter the dataframe by year 2018 first and then summarize the meat consumption by country (hint: you can use pandas `groupby` funtion for that).

In [ ]:
# start by filtering the df_per_cap dataset by year 2018


In [ ]:
# continue by summarizing meat consumption by country


**Plotly Express option**

In [ ]:
# create a bar chart


In [ ]:
# in order for bar chart to appear in descending order of meat consumption
# sort pandas dataframe by meat consumption 
# you can use sort_values method for that


In [ ]:
# create stacked bar chart

# start by pivoting the 2018 dataframe to have types of meat as columns and countries as rows


# if you want the bars to appear ordered by total meat consumption, add total column and sort the dataframe by it
# you can use the .sum() method providing the necessary axis as an argument


# drop the total column after the sorting so that it doesn't appear on the graph


# unpivot the dataframe

# create stacked bar chart


**Plotly Option**

In [ ]:
# create a bar chart


In [ ]:
# create a sorted bar chart


In [ ]:
# create stacked bar chart

# start by pivoting the 2018 dataframe to have types of meat as columns and countries as rows


# if you want the bars to appear ordered by total meat consumption, add total column and sort the dataframe by it
# you can use the .sum() method providing the necessary axis as an argument

# drop the total column after the sorting so that it doesn't appear on the graph


# create a stacked bar chart (use barmode='stack')
